# Content-based Filtering

### Loading Libraries

In [44]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

### Loading Datasets

In [45]:
movies = pd.read_csv('../../data/ml-25m/movies.csv')
print(movies.columns)

Index(['movieId', 'title', 'genres'], dtype='object')


### Titel bereinigen

In [46]:
# Extrahiere das Erscheinungsjahr aus dem Titel und füge es als neue Spalte hinzu
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)')

# Entferne das Erscheinungsjahr aus dem Titel
movies['title'] = movies['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True)

movies without year

In [47]:
# Filme ohne Erscheinungsjahr ausgeben
movies_without_year = movies[movies['year'].isna()]


print("the number of movies without year is: ", movies['year'].isna().sum())

movies_without_year.to_csv('movies_without_year.csv', index=False)
print("Movies without year saved to 'movies_without_year.csv'")

movies_with_year = pd.read_csv('movies_with_year.csv', sep=';')

# Füge die fehlenden Jahre aus der neuen CSV-Datei ein, basierend auf den Titeln
movies['year'] = movies.apply(
    lambda row: movies_with_year[movies_with_year['title'] == row['title']]['year'].values[0]
    if pd.isnull(row['year']) and row['title'] in movies_with_year ['title'].values
    else row['year'],
    axis=1
)

# Filme ohne Erscheinungsjahr ausgeben
movies_without_year1 = movies[movies['year'].isna()]

print("the number of movies without year is: ", movies['year'].isna().sum())

movies_without_year1.to_csv('movies_without_year1.csv', index=False)
print("Movies without year saved to 'movies_without_year1.csv'")


# Entferne Filme ohne Erscheinungsjahr
movies = movies.dropna(subset=['year'])
movies.isnull().sum()

print(movies.head(5))

the number of movies without year is:  410
Movies without year saved to 'movies_without_year.csv'
the number of movies without year is:  83
Movies without year saved to 'movies_without_year1.csv'
   movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                         Comedy|Drama|Romance  1995  
4                                       Comedy  1995  


### Genres aufbereiten


In [48]:
print("Anzahl der Filme mit '(no genres listed)':", movies[movies['genres'] == '(no genres listed)'].shape[0])

# Speichern der Filme mit "(no genres listed)" in eine CSV-Datei
movies_no_genres_listed = movies[movies['genres'] == '(no genres listed)']
movies_no_genres_listed.to_csv('movies_no_genres_listed.csv', index=False)

# Neue CSV-Datei mit den Genres laden
movies_with_genres = pd.read_csv('filtered_filme.csv')

# Zusammenführen der DataFrames basierend auf den Titeln und Jahren
movies_updated = pd.merge(movies, movies_with_genres[['title', 'year', 'genres']], on=['title', 'year'], how='left', suffixes=('', '_new'))

# Aktualisieren der Genres im movies DataFrame nur für die übereinstimmenden Einträge
movies_updated['genres'] = movies_updated['genres_new'].combine_first(movies_updated['genres'])

# Entferne die temporäre Spalte
movies_updated = movies_updated.drop(columns=['genres_new'])
movies = movies_updated

print("Anzahl der Filme mit '(no genres listed)':", movies[movies['genres'] == '(no genres listed)'].shape[0])

# Speichern der Filme mit "(no genres listed)" in eine CSV-Datei
movies_no_genres_listed1 = movies[movies['genres'] == '(no genres listed)']
movies_no_genres_listed1.to_csv('movies_no_genres_listed.csv', index=False)


# # Entferne "(no genres listed)" aus der Genre-Liste
# movies['genres'] = movies['genres'].replace('(no genres listed)', '')

# # Trenne die Genres in separate Listen
# genre_list = movies['genres'].str.split('|')

# # Finde alle einzigartigen Genres
# all_genres = set(genre for sublist in genre_list for genre in sublist if genre)

# # Erstelle für jedes Genre eine Spalte und fülle sie mit binären Werten
# for genre in all_genres:
#     movies[genre] = movies['genres'].apply(lambda x: int(genre in x.split('|')))

# # Entferne die ursprüngliche 'genres' Spalte
# movies = movies.drop(columns=['genres'])

# print(movies.head(5))

Anzahl der Filme mit '(no genres listed)': 5002
Anzahl der Filme mit '(no genres listed)': 1597


### Nach Null-Werten suchen

In [49]:
movies.info()
movies.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62340 entries, 0 to 62339
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62340 non-null  int64 
 1   title    62340 non-null  object
 2   genres   62340 non-null  object
 3   year     62340 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.9+ MB


movieId    0
title      0
genres     0
year       0
dtype: int64

movies without genre

### Feature Set 1: Genre

In [50]:
# Berechnung der Cosine Similarity basierend auf den Genres
genre_columns = list(all_genres)  
# genre_matrix = movies_with_tags[genre_columns].values
cosine_sim = cosine_similarity(genre_matrix, genre_matrix)

# Funktion zur Empfehlung von Filmen basierend auf Genres
def recommend_movies_by_genre(movie_title, cosine_sim=cosine_sim):
    # Index des eingegebenen Films finden
    idx = movies_with_tags[movies_with_tags['title'] == movie_title].index[0]
    
    # Ähnlichkeitsscores des Films mit allen anderen Filmen
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sortierung der Filme basierend auf der Ähnlichkeit
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Indizes der 10 ähnlichsten Filme erhalten
    sim_scores = sim_scores[1:11]
    
    # Filmtitel der 10 ähnlichsten Filme ausgeben
    movie_indices = [i[0] for i in sim_scores]
    return movies_with_tags['title'].iloc[movie_indices]



NameError: name 'genre_matrix' is not defined

### Test

In [ ]:
recommended_movies = recommend_movies_by_genre('Toy Story')
print(recommended_movies)